In [4]:
import requests

import faiss

# LM STUDIO

Это прикольная штука, которая позволяет не разворачивать модели вручную из hugging face, а пользоваться уже испытанными моделями.

Если пытаешься сам развернуть - это гемор. Нужно скачать, понять как работать с ней, установить кучу всего и тд. А так ты в studio запустил - и через запросы общаешься с ней

## Сначала надо найти и добавить модель

Советую вам для начала выбрать mistral-7b-instruct-v0.3, ибо она довольно неплохая и мало места занимает (гигов до 3-х). Как запустите и поймете - там уже можете поиграться)

![screen_model_id.png](assets/images/model_searching.png)

## После надо скачать модель

![screen_model_id.png](assets/images/btn_download_model.png)

## Выбрать модель

![screen_model_id.png](assets/images/load_model.png)

Модель будет загружена в оперативную память (тут аккуратнее, ибо память имеет тенденцию заканчиваться)

## Как должна отображаться готовая модель

![screen_model_id.png](assets/images/ready_model.png)

# Подготовка данных и функций

Нужно зафиксировать адрес и порт LM Studio, а также id модели из lm studio. На скрине ниже показано, где найти необходимую информацию в интерфейсе программы LM Studio.

![screen_model_id.png](assets/images/screen_model_id.png)

In [14]:
# Тут инфа по умолчанию
LM_ADDRESS: str = 'http://localhost'
LM_PORT: str = '1234'

MODEL_ID: str = 'mistral-7b-instruct-v0.3'

Функция для обращения к ml studio для запроса к LLM модели.
- model_id - id модели, которую необходимо взять из LM Studio
- prompt - запрос от пользователя к LLM модели (ниже будет пример видео)
- temperature - чем ниже значение температуры, тем более детерминированными будут результаты в смысле того, что будет выбран самый вероятный следующий токен.
    Увеличение температуры может привести к большей случайности, что способствует более разнообразным или творческим результатам.
- max_tokens - максимальное количество токенов. Чем меньше - тем короче и локоничнее будет ответ (в теории)<br>
-1 - это без ограничений

In [6]:
def request_to_llm(model_id: str, prompt: str, temperature: float = 0.7, max_tokens: int = -1) -> dict[str, str]:
    headers = {
        'Content-Type': 'application/json',
    }

    json_data = {
        'model': model_id,
        'messages': [
            {
                'role': 'user',
                'content': prompt,
            },
        ],
        'temperature': temperature,
        'max_tokens': max_tokens,
        'stream': False,
    }
    
    response = requests.post(f'{LM_ADDRESS}:{LM_PORT}/v1/chat/completions', headers=headers, json=json_data)
    return response.json()['choices'][0]['message']

Наша "база данных", которую я выгрузил из рассписания ЗМШ. В кавычках - так как это еще не прям база данных, а данные для базы данных<br>

Тут в одной строке идут 4 строки (\n как раделитель):
- когда выступает
- название лекции
- кто выступает
- описание лекции

In [7]:
data = [
    "Среда, 29 января.\nАксиома выбора и оргвопросы.\nВихарев Сергей Викторович\n",
    "Среда, 29 января.\nФормула Эйлера и защищённая передача информации: как работает алгоритм RSA\nСавватеев Алексей Владимирович\nОдин из самых распространённых алгоритмов секретной передачи данных, называемый RSA, содержит в своём ядре очень красивую и нетривиальную, хоть и абсолютно школьную, математику. Я про все тонкости и секреты расскажу на лекции, опуская некоторые детали! По сути, речь идёт о тонких свойствах систем остатков от деления на «полупростые» числа, являющиеся произведением двух простых.",
    "Четверг, 30 января.\nРаскраски гиперграфов и вероятностный метод в комбинаторике\nРайгородский Андрей Михайлович\nГиперграф — это важный объект комбинаторики, который обобщает понятие графа. У него тоже есть вершины, но рёбра могут состоять не из двух вершин, а из большего их числа. Например, вершинами могут быть авторы научных статей, а рёбрами — наборы авторов тех или иных конкретных работ. Вершины гиперграфа, как и вершины графа, можно красить. Раскраска называется правильной, если каждое ребро в ней неодноцветно. Мы поговорим об условиях, при которых гиперграфы допускают правильную раскраску в 2 цвета.",
    "Четверг, 30 января.\nСвязные подмножества плоскости\nЛипин Антон\nЧто такое связность с точки зрения топологии? Верно ли, что любые две точки связного множества можно соединить кривой, которая содержится в этом множестве? Что такое компоненты связности, и чем они отличаются от квазикомпонент? И сколько точек необходимо удалить из бесконечного связного множества, чтобы остаток рассыпался на одноточечные компоненты?",
    "Четверг, 30 января.\nКак стать гением, если ты маньяк?\nБубб Полина Дмитриевна\nЖизнь не так гармонична, как может показаться, главное уметь использовать эту дисгармонию и непостоянство в свое благо",
    "Четверг, 30 января.\nИИ в создании и визуализации танцев\nБоярских Арина Алексеевна\nРазберёмся, как ИИ понимает и генерирует хореографию, где это может быть применимо.",
    "Пятница, 31 января.\nАлгоритмическая торговля: просто о сложном. Краткий обзор и пример применения\nАгафонов Максим Сергеевич\nХотите узнать, как современные технологии позволяют автоматизировать торговые процессы и получать преимущество на финансовых рынках? Приходите на доклад.",
    "Пятница, 31 января.\nТест Миллера — Рабина\nОвчинников Данила Сергеевич\nСлышали про простые числа? Мне кажется, вы не умеете в быстрое определение числа на простоту.",
    "Пятница, 31 января.\nЭнигма и бомба Тьюринга — Уэлшмана, или Как математики выиграли войну\nМитяшин Тимофей Витальевич\nРасскажу про маленький деревянный ящик и про то, как его криптоанализ повлиял на мировую историю.",
    "Пятница, 31 января.\nПро Надёжность\nЗырянов Константин Александрович\nМатематическая лекция про мечты, старение, инверсию, ракеты и самосвалы, алкоголь, мобильные приложения, падших женщин, фашистов, какие-то схемы, выживание, первокурсников, Бэтмена, Богатство, канцелярские приборы, Дунаева и негров...",
    "Пятница, 31 января.\nДжазовая лекция\nЛехтик Владислав Анатольевич\n",
    "Суббота, 1 февраля.\nМатематика и физика коллективных эффектов в ансамблях взаимодействующих магнитных наночастиц\n🌂 Иванов Алексей Олегович\nНа лекции поговорим о том, как применять знания из математического анализа, алгебры, дифференциальных уравнений для создания математических моделей, которые помогают понять, как ведут себя магнитные наночастицы. Рассмотрим, насколько интересным бывает использование наночастиц в современных технологиях и биомедицине. Также кратко обсудим теоретические результаты, успешно прогнозирующие свойства магнитных наносуспензий.",
    "Суббота, 1 февраля.\nДилемма (?) заключённого\nКишун Кирилл\nНебольшое погружение в теорию игр.",
    "Суббота, 1 февраля.\nОткрытые API по-русски\nЧерных Сергей Николаевич\nРасскажу, что такое открытые API, открытый банкинг и что по этому поводу думает ЦБ.",
    "Суббота, 1 февраля.\nКаждый охотник желает знать, где в радуге розовый\nКузнецов Алексей Сергеевич\nИз доклада мы узнаем:a) где в радуге розовый;b) как фотоны проникают в наш мозг;c) что от нас скрывают производители мониторов.",
    "Суббота, 1 февраля.\n3D-печать — это круто!\nПлаксин Алексей Дмитриевич\nИз доклада вы узнаете, как появились 3D-принтеры, как развивалась история и какая занимательная внутри математика. А ещё сможете пощупать!",
]

# Блок инициализации базы данных

In [8]:
from sentence_transformers import SentenceTransformer

/Users/ivanhodnenko/miniforge3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Используем модель для получения эмбеддингов
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # тут она сама скачать модель должна (нужен интернет)
embeddings = model.encode(data)

Инициализируем нашу базу данных с простейшим линейным индексом

In [10]:
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance для плоского индекса
index.add(embeddings)  # Добавляем эмбеддинги в индекс

Эксперимент, что информация в бд ищется корректно

In [11]:
# Пример: поиск ближайших соседей для запроса
query_text = "Кто читает лекцию про джаз?"
query_vector = model.encode([query_text]).astype('float32')

k = 3  # Количество ближайших соседей
distances, indices = index.search(query_vector, k)

# Печатаем результаты
for i in range(k):
    print(f"Результат {i+1}: {data[indices[0][i]]} (дистанция: {distances[0][i]})\n")

Результат 1: Пятница, 31 января.
Про Надёжность
Зырянов Константин Александрович
Математическая лекция про мечты, старение, инверсию, ракеты и самосвалы, алкоголь, мобильные приложения, падших женщин, фашистов, какие-то схемы, выживание, первокурсников, Бэтмена, Богатство, канцелярские приборы, Дунаева и негров... (дистанция: 27.537193298339844)

Результат 2: Четверг, 30 января.
Как стать гением, если ты маньяк?
Бубб Полина Дмитриевна
Жизнь не так гармонична, как может показаться, главное уметь использовать эту дисгармонию и непостоянство в свое благо (дистанция: 27.835704803466797)

Результат 3: Четверг, 30 января.
Связные подмножества плоскости
Липин Антон
Что такое связность с точки зрения топологии? Верно ли, что любые две точки связного множества можно соединить кривой, которая содержится в этом множестве? Что такое компоненты связности, и чем они отличаются от квазикомпонент? И сколько точек необходимо удалить из бесконечного связного множества, чтобы остаток рассыпался на одното

# Вот тут уже прям обращение к LLM с получением результата

In [13]:
# Наш запрос
prompt = "Про что читает лекцию Константин Зырьянов?"

# получение из базы данных релевантной информации
distances, indices = index.search(query_vector, 1)
db_record = data[indices[0][0]]
print('Найденная релевантная информация:', db_record)

context = (
    f"Есть информация: {db_record}\n\n. "
    "На каждой новой строке находятся: "
    "дата, название, автор и описание. "
    f"Найди из этой информации {prompt}"
)

request_to_llm(MODEL_ID, context)

Найденная релевантная информация: Пятница, 31 января.
Про Надёжность
Зырянов Константин Александрович
Математическая лекция про мечты, старение, инверсию, ракеты и самосвалы, алкоголь, мобильные приложения, падших женщин, фашистов, какие-то схемы, выживание, первокурсников, Бэтмена, Богатство, канцелярские приборы, Дунаева и негров...


{'role': 'assistant',
 'content': ' По данной информации, Константин Зырянов читает лекцию про тему "Мечты, старение, инверсию, ракеты и самосвалы, алкоголь, мобильные приложения, падших женщин, фашистов, какие-то схемы, выживание, первокурсников, Бэтмена, Богатство, канцелярские приборы, Дунаева и негров".'}